In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import scipy
# 不用print，直接显示结果
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 显示所有列
pd.set_option('display.max_columns', 600)

## 【练习一】 现有一份关于某超市牛奶销售额的时间序列数据，请完成下列问题：
### （a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）
### （b）计算除去春节、国庆、五一节假日的月度销售总额
### （c）按季度计算周末（周六和周日）的销量总额
### （d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和
### （e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）

In [34]:
MilkSale = pd.read_csv('data/time_series_one.csv', parse_dates=['日期'])
MilkSale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   日期      1000 non-null   datetime64[ns]
 1   销售额     1000 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 15.8 KB


In [5]:
MilkSale.head()

,日期,销售额
0,2017/2/17,2154
1,2017/2/18,2095
2,2017/2/19,3459
3,2017/2/20,2198
4,2017/2/21,2413


In [26]:
# （a）销售额出现最大值的是星期几？（提示：利用dayofweek函数）
## 将日期列类型改为日期型
MilkSale[MilkSale['销售额'] == MilkSale['销售额'].max()]['日期'].dt.dayofweek

926    6
Name: 日期, dtype: int64

In [45]:
#（b）计算除去春节、国庆、五一节假日的月度销售总额
## 找出节假日排除
MilkSale['日期'].min(),MilkSale['日期'].max()
holiday = pd.date_range(start='20170501', end='20170503').append(pd.date_range(start='20171001', end='20171007')).append(
          pd.date_range(start='20180215', end='20180221')).append(pd.date_range(start='20180501', end='20180503')).append(
          pd.date_range(start='20181001', end='20181007')).append(pd.date_range(start='20190204', end='20190224')).append(
          pd.date_range(start='20190501', end='20190503')).append(pd.date_range(start='20191001', end='20191007'))

MilkSale[~MilkSale['日期'].isin(holiday)].set_index('日期').resample('MS').sum()

(Timestamp('2017-02-17 00:00:00'), Timestamp('2019-11-13 00:00:00'))

,销售额
日期,
2017-02-01,31740
2017-03-01,80000
2017-04-01,74734
2017-05-01,76237
2017-06-01,80750
2017-07-01,83107
2017-08-01,85715
2017-09-01,79604
2017-10-01,61197


In [44]:
#（c）按季度计算周末（周六和周日）的销量总额
MilkSaleOfWeekend = MilkSale[(MilkSale['日期'].dt.dayofweek == 6) | (MilkSale['日期'].dt.dayofweek == 5)]
MilkSaleOfWeekend.set_index('日期').resample('QS').sum()

,销售额
日期,
2017-01-01,32894
2017-04-01,66692
2017-07-01,69099
2017-10-01,70384
2018-01-01,74671
2018-04-01,69950
2018-07-01,74245
2018-10-01,74699
2019-01-01,77835


In [ ]:
# （d）从最后一天开始算起，跳过周六和周一，以5天为一个时间单位向前计算销售总和


In [ ]:
# （e）假设现在发现数据有误，所有同一周里的周一与周五的销售额记录颠倒了，请计算2018年中每月第一个周一的销售额（如果该周没有周一或周五的记录就保持不动）

## 【练习二】 继续使用上一题的数据，请完成下列问题：
### （a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）
### （b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果
### （c）将(c)中的“向前5天”改为“向前非周末5天”，请再次计算结果

In [48]:
# （a）以50天为窗口计算滑窗均值和滑窗最大值（min_periods设为1）
MilkSale.set_index('日期').rolling(window=50,min_periods=1).mean()
MilkSale.set_index('日期').rolling(window=50,min_periods=1).max()

,销售额
日期,
2017-02-17,2154.000000
2017-02-18,2124.500000
2017-02-19,2569.333333
2017-02-20,2476.500000
2017-02-21,2463.800000
...,...
2019-11-09,2955.860000
2019-11-10,2962.060000
2019-11-11,2988.780000


,销售额
日期,
2017-02-17,2154.0
2017-02-18,2154.0
2017-02-19,3459.0
2017-02-20,3459.0
2017-02-21,3459.0
...,...
2019-11-09,4048.0
2019-11-10,4048.0
2019-11-11,4048.0


In [83]:
#（b）现在有如下规则：若当天销售额超过向前5天的均值，则记为1，否则记为0，请给出2018年相应的计算结果
MilkSale_rolling = MilkSale.copy()
MilkSale_rolling = pd.merge(MilkSale_rolling, MilkSale_rolling.set_index('日期').rolling(window=5,min_periods=1).mean().reset_index(), how='left', on=['日期'])
MilkSale_rolling
MilkSale_rolling.loc[MilkSale_rolling['销售额_x'] >= MilkSale_rolling['销售额_y'],'C'] = '1'
MilkSale_rolling.loc[MilkSale_rolling['销售额_x'] < MilkSale_rolling['销售额_y'],'C'] = '0'
MilkSale_rolling.set_index('日期').loc[pd.date_range(start='20180101',end='20181231'),:]

,日期,销售额_x,销售额_y
0,2017-02-17,2154,2154.000000
1,2017-02-18,2095,2124.500000
2,2017-02-19,3459,2569.333333
3,2017-02-20,2198,2476.500000
4,2017-02-21,2413,2463.800000
...,...,...,...
995,2019-11-09,3022,2860.600000
996,2019-11-10,2961,2939.000000
997,2019-11-11,3984,3213.800000
998,2019-11-12,2799,3179.800000


,销售额_x,销售额_y,C
2018-01-01,2863,2731.6,1
2018-01-02,2324,2699.6,0
2018-01-03,2278,2688.4,0
2018-01-04,2261,2505.8,0
2018-01-05,2250,2395.2,0
...,...,...,...
2018-12-27,2505,2822.0,0
2018-12-28,3171,2725.8,1
2018-12-29,2642,2716.2,0
2018-12-30,2597,2727.0,0
